In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup as bsoup

import constants

In [12]:
test_run = True

ukbb_data_file = "small_ukbiobank.csv" if test_run else "ukbiobank.csv"
ukbb_data_path = os.path.join(constants.UK_BIOBANK_DATA_PATH, ukbb_data_file)

ukbb_index_html_path = os.path.join(constants.UK_BIOBANK_DATA_PATH, "ukbiobank.html")
ukbb_index_csv_path = os.path.join(constants.UK_BIOBANK_DATA_PATH, "ukbiobank_index.csv")

In [26]:
if os.path.exists(ukbb_index_csv_path):
    ukbb_index = pd.read_csv(ukbb_index_csv_path)
else:
    ukbb_html = bsoup(open(ukbb_index_html_path,'r').read())
    ukbb_index_html = ukbb_html.find_all("table")[1]
    ukbb_index = pd.read_html(str(ukbb_index_html))[0]
    ukbb_index.columns = [col.lower() for col in ukbb_index.columns]
    ukbb_index.to_csv(ukbb_index_csv_path, index=False)

In [27]:
ukbb_index.head()

,column,udi,count,type,description
0,0,eid,502543,Sequence,Encoded anonymised participant ID
1,1,23-0.0,456606,Categorical (single),Spirometry methodUses data-coding 100270 compr...
2,2,23-1.0,18135,Categorical (single),Spirometry methodUses data-coding 100270 compr...
3,3,23-2.0,26790,Categorical (single),Spirometry methodUses data-coding 100270 compr...
4,4,31-0.0,502543,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...


In [28]:
ukbb_data = pd.read_csv(ukbb_data_path, low_memory=False)

In [31]:
ukbb_data.count()

eid          9999
23-0.0       9056
23-1.0        363
23-2.0        528
31-0.0       9999
             ... 
41247-0.3      13
41247-0.4       2
41247-0.5       1
41247-0.6       0
41252-0.0    7865
Length: 3973, dtype: int64